Importing the dependencies

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow as tf

Importing the input files for X and Y

In [ ]:
x_vals = np.load("X2.npy")
y_vals = np.load("Y2.npy")

Seed initialization

seed = 0
tf.set_random_seed(seed)
np.random.seed(seed)

Data requirements (placeholders) so that the data is added in the graph

In [ ]:
x_data = tf.placeholder(shape=[None, len(products)], dtype=tf.float32)
y_data = tf.placeholder(shape=[None, len(products)], dtype=tf.float32)
learningRate = tf.placeholder(dtype=tf.float32)

Assigning training and validation (test) data set

In [ ]:
train_indices = np.random.choice(len(x_vals), int(len(x_vals) * 0.8), replace=False)
test_indices = np.array(list(set(range(len(x_vals))) - set(train_indices)))

x_vals_train = x_vals[train_indices]
x_vals_test = x_vals[test_indices]
y_vals_train = y_vals[train_indices]
y_vals_test = y_vals[test_indices]

Setting the hidden layers (an example of 5) <br>
Ensuring that the matrix dimensions are consistent for matrix multiplication

Initialization for the variables is based on the values suggested in Deep Learning Book by Ian Goodfellow<br>
The variables ar initialized with uniform random

In [ ]:
h1 = 80
h2 = 80
h3 = 80
h4 = 80
h5 = 20

m = 6

A1 = tf.Variable(tf.random_uniform(shape=[p,h1],maxval=tf.sqrt(m/(p+h1+0.0)),minval=-tf.sqrt(m/(p+h1+0.0))))
b1 = tf.Variable(tf.constant([0.0]*h1))
A2 = tf.Variable(tf.random_uniform(shape=[h1,h2],maxval=tf.sqrt(m/(h1+h2+0.0)),minval=-tf.sqrt(m/(h1+h2+0.0))))
b2 = tf.Variable(tf.constant([0.0]*h2))
A3 = tf.Variable(tf.random_uniform(shape=[h2,h3],maxval=tf.sqrt(m/(h2+p+0.0)),minval=-tf.sqrt(m/(h2+p+0.0)))) #tf.Variable(tf.random_normal(shape=[h2,h3],stddev= 2))
b3 = tf.Variable(tf.constant([0.0]*h3))
A4 = tf.Variable(tf.random_uniform(shape=[h3,h4],maxval=tf.sqrt(m/(h3+h4+0.0)),minval=-tf.sqrt(m/(h3+h4+0.0)))) #tf.Variable(tf.random_normal(shape=[h2,h3],stddev= 2))
b4 = tf.Variable(tf.constant([0.0]*h4))
A5 = tf.Variable(tf.random_uniform(shape=[h4,h5],maxval=tf.sqrt(m/(h4+h5+0.0)),minval=-tf.sqrt(m/(h4+h5+0.0)))) #tf.Variable(tf.random_normal(shape=[h2,h3],stddev= 2))
b5 = tf.Variable(tf.constant([0.0]*h5))
A6 = tf.Variable(tf.random_uniform(shape=[h5,p],maxval=tf.sqrt(m/(h5+p+0.0)),minval=-tf.sqrt(m/(h5+p+0.0)))) #tf.Variable(tf.random_normal(shape=[h2,h3],stddev= 2))
b6 = tf.Variable(tf.constant([0.0]*p))

Setting up the activation functions for different layers

In [ ]:
output1 = tf.nn.relu6(tf.add(tf.matmul(x_data, A1),b1))
output2 = tf.nn.relu6(tf.add(tf.matmul(output1, A2),b2))
output3 = tf.nn.relu6(tf.add(tf.matmul(output2, A3),b3))
output4 = tf.nn.relu6(tf.add(tf.matmul(output3, A4),b4))
output5 = tf.nn.relu6(tf.add(tf.matmul(output4, A5),b5))
output6 = tf.nn.sigmoid(tf.add(tf.matmul(output5, A6),b6))

Setting the loss function as simple squared loss

In [ ]:
loss = tf.reduce_mean(tf.squared_difference(output6, y_data))

Setting the dropouts for regularization

In [ ]:
d1 = 0.2
d2 = 0.2
d3 = 0.2
d4 = 0.2
d5 = 0.2
d6 = 0.2

kp1   = tf.placeholder(dtype=tf.float32)
drop1 = tf.nn.dropout(output1,kp1)
kp2   = tf.placeholder(dtype=tf.float32)
drop2 = tf.nn.dropout(output2,kp2)
kp3   = tf.placeholder(dtype=tf.float32)
drop3 = tf.nn.dropout(output3,kp3)
kp4   = tf.placeholder(dtype=tf.float32)
drop4 = tf.nn.dropout(output4,kp4)
kp5   = tf.placeholder(dtype=tf.float32)
drop5 = tf.nn.dropout(output5,kp5)
kp6   = tf.placeholder(dtype=tf.float32)
drop6 = tf.nn.dropout(output6,kp6)

Running the code from previously saved state (do not initialize)

In [ ]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
#sess.run(init)
saver.restore(sess, "/home/ashutosh/Desktop/instacart/model_non_lstm_relu.ckpt")

Running the model for user defined number of iterations<br>
Changing the learning rate over time

In [ ]:
epoch = 1
r_rate = 0.00005

for i in range(iterations):
    batch_size = 64#int(np.random.uniform(16,32,1))
    # Choose random indices for batch selection
    rand_index = np.random.choice(len(x_vals_train), size=batch_size)
    # Get random batch
    rand_x = x_vals_train[rand_index]
    rand_y = y_vals_train[rand_index]

    if i%1000 == 0:
        epoch += 0.05
        rate = r_rate/epoch

    # Run the training step
    sess.run(train_step, feed_dict={x_data: rand_x, y_data:rand_y, learningRate: rate, kp1:d1, kp2:d2, kp3:d3, kp4:d4})

    # Get and store the train loss
    temp_loss = sess.run(loss, feed_dict={x_data: x_vals_train, y_data: y_vals_train})
    loss_vec.append(temp_loss)
    # Get and store the test loss
    test_temp_loss = sess.run(loss, feed_dict={x_data: x_vals_test, y_data: y_vals_test})
    test_loss.append(test_temp_loss)